In [1]:
# Fine-tuning: https://platform.openai.com/docs/guides/fine-tuning

import json
import os
import openai
import requests

In [2]:
key_location = '/Users/silvi/Downloads/key-location/genaikey.txt'

with open(key_location, 'r') as file:
    key = file.readline().strip()

In [3]:
openai.api_key = key

In [4]:
with open("./data.jsonl") as f: # The with keyword automatically closes the file when you are done
   print(f.read())

{"messages": [{"role": "system", "content": "Albert is a Particle Physics Professor."}, {"role": "user", "content": "What are quarks?"}, {"role": "assistant", "content": "Quarks are fundamental particles that are the building blocks of hadrons, such as protons and neutrons. They possess fractional electric charges and come in six types: up, down, charm, strange, top, and bottom."}]}
{"messages": [{"role": "system", "content": "Albert is a Particle Physics Professor."}, {"role": "user", "content": "What are leptons?"}, {"role": "assistant", "content": "Leptons are another type of fundamental particle, which includes electrons, muons, and tau particles. They do not experience the strong nuclear force and are not composed of quarks."}]}
{"messages": [{"role": "system", "content": "Albert is a Particle Physics Professor."}, {"role": "user", "content": "What is the Higgs Boson?"}, {"role": "assistant", "content": "The Higgs boson is a fundamental particle associated with the Higgs field, wh

In [5]:
file = openai.files.create(
  file=open("data.jsonl", "rb"),
  purpose="fine-tune"
)

In [6]:
openai.fine_tuning.jobs.create(
  training_file=file.id,
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-0QefUSFQ1RulkIN9yOjtxTNp', created_at=1709155887, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-yWjH9ZcfOsQiCyzJVDdBx0JV', result_files=[], status='validating_files', trained_tokens=None, training_file='file-OFiYN2VyCiHrmPCt0Uy5gpIh', validation_file=None, user_provided_suffix=None)


In [11]:
# List 10 fine-tuning jobs
openai.fine_tuning.jobs.list(limit=10)


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-0QefUSFQ1RulkIN9yOjtxTNp', created_at=1709155887, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-yWjH9ZcfOsQiCyzJVDdBx0JV', result_files=[], status='running', trained_tokens=None, training_file='file-OFiYN2VyCiHrmPCt0Uy5gpIh', validation_file=None, user_provided_suffix=None), FineTuningJob(id='ftjob-TocBOGAPFhlhp44Z8BerpFnb', created_at=1709154116, error=Error(code='invalid_n_examples', message='Training file has 2 example(s), but must have at least 10 examples', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-yWjH9ZcfOsQ

In [21]:
# Retrieve the state of a fine-tune
fine_tuning_job_state = openai.fine_tuning.jobs.retrieve("ftjob-0QefUSFQ1RulkIN9yOjtxTNp")

print("fine-tuning job status", fine_tuning_job_state.status)
print("fine-tuning job error", fine_tuning_job_state.error)
# Eventually, fine_tuned_model gets from None to ft:gpt-3.5-turbo:my-org:custom_suffix:id
print("fine-tuning model", fine_tuning_job_state.fine_tuned_model)

# Cancel a job, if it takes too long...
# openai.fine_tuning.jobs.cancel("ftjob-0QefUSFQ1RulkIN9yOjtxTNp")

fine-tuning job status succeeded
fine-tuning job error Error(code=None, message=None, param=None, error=None)
fine-tuning model ft:gpt-3.5-turbo-0613:bitdrops-ai::8xLv0NuE


In [25]:
completion = openai.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:bitdrops-ai::8xLv0NuE",
  messages=[
    {"role": "system", "content": "Albert is a Particle Physics Professor."},
    {"role": "user", "content": "What are quarks?"}
  ],
  temperature=0.7 # how stocastic on a scale of 0 to 2; suggested 0.7
)
print(completion.choices[0].message.content)

# related training was:
# {"messages": [{"role": "system", "content": "Albert is a Particle Physics Professor."}, 
# {"role": "user", "content": "What are quarks?"}, 
# {"role": "assistant", "content": "Quarks are fundamental particles that are the building blocks of hadrons, such as protons and neutrons. They possess fractional electric charges and come in six types: up, down, charm, strange, top, and bottom."}]}


Quarks are fundamental particles that are the building blocks of hadrons, such as protons and neutrons. They possess fractional electric charges and come in six types: up, down, charm, strange, top, and bottom. Quarks are bound together by the strong nuclear force mediated by gluons.
